In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFECV, SelectKBest, RFE, f_regression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.inspection import PartialDependenceDisplay
import joblib

In [ ]:
model_name = 'Linear_regression_model'
model = LinearRegression()

In [ ]:
# read data


In [ ]:
y = df.pop('Hsig (m)')
meta = pd.concat([df.pop(x) for x in ['time_utc','station_ind']], axis=1)
X = df

In [ ]:
X.shape

In [ ]:
n_features_to_select = [2, 5, 9]

In [ ]:
X_train, X_test, y_train, y_test, meta_train, meta_test = train_test_split(X, y, meta, test_size=0.25, random_state=42)
X_train

# Model pipeline and grid search parameters

In [ ]:
pipeline_steps = [
    ('scaler', StandardScaler()),
    ('feature_selection', RFECV(estimator=GradientBoostingRegressor(n_estimators=50), cv=5)),
    ('estimator', model)
]
pipeline = Pipeline(pipeline_steps)

In [ ]:
# Define model parameters for GridSearchCV as a python dictionary with key as the parameter name and value as a list of values to try
model_params = { 'estimator__fit_intercept': [True, False]}

In [ ]:
# Define different experiments with different pipeline parameters
# For example first does not use any feature selection "passthrough", second uses RFE and third KBest
# To every experiment I add **modelparams to try different model parameters as well
param_grid = [
    {
        'scaler': [StandardScaler(), MinMaxScaler(), 'passthrough'],
        'feature_selection': ['passthrough'],
        'estimator': [model],
        **model_params,
    },
    {
        'scaler': [StandardScaler(), MinMaxScaler(), 'passthrough'],
        'feature_selection': [SelectKBest(f_regression)],
        'feature_selection__k': n_features_to_select,
        'estimator': [model],
        **model_params,
    }
]

# Searching best model and feature importance

In [ ]:
# Perform GridSearchCV to find the best parameters with cross validation
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_absolute_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
# Print the best parameters and score
print(f'Best parameters for {model_name} for Hsig (m): {grid_search.best_params_}')
print(f'Best score for {model_name} for Hsig (m): {grid_search.best_score_}')

In [ ]:
# Predicting the model on test set
y_pred_test = grid_search.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

In [ ]:
if hasattr(grid_search.best_estimator_['estimator'], 'feature_importances_'):
    print("Feature importances:")
    print(grid_search.best_estimator_['estimator'].feature_importances_)

In [ ]:
if hasattr(grid_search.best_estimator_['estimator'], 'coef_'):
    print("Coefficients:")
    print(grid_search.best_estimator_['estimator'].coef_)

In [ ]:
if hasattr(grid_search.best_estimator_['estimator'], 'feature_importances_'):
    feature_importances = grid_search.best_estimator_['estimator'].feature_importances_
    feature_importances_df = pd.DataFrame(feature_importances, index=grid_search.best_estimator_[:-1].get_feature_names_out(), columns=['importance'])


In [ ]:
if hasattr(grid_search.best_estimator_['estimator'], 'coef_'):
    intercept = grid_search.best_estimator_['estimator'].intercept_
    coefficients = grid_search.best_estimator_['estimator'].coef_

    # Save coefficients and intercept together
    coefficients_df = pd.DataFrame({'feature': grid_search.best_estimator_[:-1].get_feature_names_out(), 'coefficient': coefficients})
    if grid_search.best_params_['estimator__fit_intercept']:
        coefficients_df.loc[-1] = ['intercept', intercept]  # Add intercept as a row
        coefficients_df.index = coefficients_df.index + 1  # Shift index
    coefficients_df = coefficients_df.sort_index()     # Sort so intercept is first


In [ ]:
coefficients_df

In [ ]:
best_pipe = grid_search.best_estimator_
selector = best_pipe.named_steps["feature_selection"]

if selector == "passthrough":
    selected_names = X_train.columns
else:
    selected_mask = selector.get_support()
    selected_names = X_train.columns[selected_mask]

# Now choose from selected_names
features_to_plot = selected_names[:6].tolist()  # or pick specific names you care about

fig = plt.figure(figsize=(10, 5))
PartialDependenceDisplay.from_estimator(
    best_pipe, 
    X_train, 
    features=features_to_plot,
    kind="individual",         # or "individual" for Individual Conditional Expectation (ICE) curves, or "both"
    grid_resolution=50)
plt.suptitle("LinRegression Partial Dependence")

In [ ]:
# Print test metrics
print("Test Metrics")
print(f"Mean Squared Error: {mse_test:.3f}")
print(f"Mean Absolute Percentage Error: {mape_test:.3f}")
print(f"R2 Score: {r2_test:.3f}")
print(f"Correlation coefficient: {(np.corrcoef(y_test, y_pred_test))[0, 1]:.3f}")

# Plot predictions against true values; plot errors

In [ ]:
plt.scatter(y_test, y_pred_test, c=meta_test['station_ind'])
plt.xlabel('Measured Hsig (m)')
plt.ylabel('Estimated Hsig (m)')
plt.ylim([0, 2])
plt.xlim([0, 2])
plt.grid()

In [ ]:
errors = y_test - y_pred_test
errors

In [ ]:
plt.hist(errors)

# Using the model for spatial case

In [ ]:
from scipy.io import readsav
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import glob
import os
import pandas as pd

In [ ]:
data_dir = '/home/study/radar_data'

lat_dict = readsav(f"{data_dir}/auxiliary/radar_lat.sav")
lon_dict = readsav(f"{data_dir}/auxiliary/radar_lon.sav")
inci_dict = readsav(f"{data_dir}/auxiliary/inci_arr.sav")
mask_dict = readsav(f"{data_dir}/auxiliary/radar_mask.sav")

lat_arr = np.flip(lat_dict['test_lat'], 0)
lon_arr = lon_dict['test_lon']
inci_arr = inci_dict['inci_arr']
mask_arr = np.flip(mask_dict['mask'], 0)

In [ ]:
file = f'{data_dir}/radar/20161026T120002+0300.png'

In [ ]:
img = np.array(Image.open(file))

# Entire day $H_S$ field

In [ ]:
files = sorted(glob.glob(f'{data_dir}/radar/20161026T*.png'))